In [59]:
from openai import OpenAI

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy import stats
import json
from itertools import combinations
import time
import random
import os
from dotenv import load_dotenv

We are trying to establish what could be going wrong with the docker container by breaking the functions in my module apart then testing them seperatly. After that we will test the fast api call and make sure no errors are still occuring. Finally the corret functions and api call will be put into a docker container

In [ ]:
# Load the environment variables from .env file
#use the .env file to not expose our secret API key
#load_dotenv()
#client = OpenAI(
    # This is the default and can be omitted
#    api_key=os.environ.get("OPENAI_API_KEY"),
#)

In [13]:
dotenv_path = os.path.join(os.path.dirname('lim-stress-tes'), '.env')
load_dotenv(dotenv_path)
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

The first function is called get_openai_responses. It takes in a prompt and will obtain responses. The number of responses can be controlled by the num_responses. To limit the tokens set the varaible max_token. Initial_delay,exponetial_base are used when a rate limit occurs and it emplements exponetial backing. The function is set to retry 10 times if a rate limit error occurs. Adjust if need be

***to test I set the num_responses = 5

In [14]:
def get_openai_responses(prompt, num_responses=40, max_tokens=100, initial_delay: float = 1,
                         exponential_base: float = 2, jitter: bool = True):
    
    """
    Retrieves a specified number of responses from OpenAI's GPT model.
    
    Args:
        prompt (str): The prompt to send to the model.
        num_responses (int): The number of responses to retrieve.
        max_tokens (int): The maximum number of tokens per response.
        initial_delay (float): Initial delay in seconds before retrying after a rate limit error.
        exponential_base (float): The base for the exponential backoff calculation.
        jitter (bool): Whether to add random jitter to the delay.

    Returns:
        list: A list of responses from the AI model.
    """
    if num_responses > 100:  # Example threshold
        raise ValueError("num_responses is too high. Please reduce the number.")

    delay = initial_delay
    responses = []
    retries = 0
    max_retries = 10

    while len(responses) < num_responses and retries < max_retries:
        try:
            AI_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that provides recommendations."},
                    {"role": "user", "content": prompt},
                ],
                max_tokens=max_tokens,
                n=1,  # Number of completions to generate
            )
            #print(AI_response)
            #need to eventually un comment these
            response_content = AI_response.choices[0].message.content
            responses.append(response_content)
        except Exception as e:
            print(f"Encountered an error: {e}")
            if 'rate limit' in str(e).lower():
                delay *= exponential_base * (1 + jitter * random.random())
                print(f"Rate limit exceeded. Waiting for {delay} seconds.")
                time.sleep(delay)
            else:
                print("Encountered a non-rate-limit error. Retrying...")
            retries += 1
            time.sleep(initial_delay)  # Basic delay for non-rate-limit errors

    if retries == max_retries:
        print("Max retries reached. Some responses may not have been retrieved.")

    return responses

The next function is called get_bert_embedding. As the name states this function will obtain the bertembeddings 

In [15]:
def get_bert_embedding(my_text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    tokens = tokenizer(my_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**tokens)
    last_hidden_states = outputs.last_hidden_state
    embedding = torch.mean(last_hidden_states, dim=1)
    
    # Convert to numpy array and check shape
    embedding_np = embedding.numpy()
    #print(f"Embedding shape: {embedding_np.shape}")  # For debugging
    return embedding_np.tolist()

The last function in my module is process_prompt. This function will compare the cosine similarities for each response and compare each response to everyother response created. It will then return the quant scores

In [ ]:
#DO Not USE

def process_prompt(prompt):
    result_data = []  # Store quant scores for all prompts
    responses = get_openai_responses(prompt)

    prompt_embedding = get_bert_embedding(prompt)
    response_embeddings = [get_bert_embedding(response) for response in responses]

    cosine_sim_scores = []
    embedding_pairs = list(combinations([prompt_embedding] + response_embeddings, 2))
    for pair in embedding_pairs:
        score = cosine_similarity([pair[0]], [pair[1]])[0][0]
        cosine_sim_scores.append(score)

    mean_score = np.mean(cosine_sim_scores)
    median_score = np.median(cosine_sim_scores)
    mode_score = stats.mode(cosine_sim_scores)
    average_score = np.average(cosine_sim_scores)
    sample_size = len(cosine_sim_scores)

    quant_scores = {
        "mean": mean_score,
        "median": median_score,
        "mode": mode_score,
        "average": average_score,
        "sample_size": sample_size
    }

    result_data.append({"prompt": prompt, "quant_scores": quant_scores})

    return result_data

In [42]:
#original DO NOT USE

def process_prompt(prompt):
    result_data = []  # Store quant scores for all prompts
    responses = get_openai_responses(prompt, num_responses=5)

    prompt_embedding = get_bert_embedding(prompt)
    response_embeddings = [get_bert_embedding(response) for response in responses]

    cosine_sim_scores = []
    for response_emb in response_embeddings:
        # Compute cosine similarity between prompt and each response
        # Since both are 2D arrays, no need to wrap them in another list
        score = cosine_similarity(prompt_embedding, response_emb)[0][0]
        cosine_sim_scores.append(score)
    mean_score = np.mean(cosine_sim_scores)
    median_score = np.median(cosine_sim_scores)
    mode_result=stats.mode(cosine_sim_scores)
    mode_score = mode_result.mode[0]
    average_score = np.average(cosine_sim_scores)
    sample_size = len(cosine_sim_scores)

    quant_scores = {
        "mean": mean_score,
        "median": median_score,
        "mode": mode_score,
        "average": average_score,
        "sample_size": sample_size
    }

    result_data.append({"prompt": prompt, "quant_scores": quant_scores})
    return result_data


In [56]:
#updated and unbugged

def process_prompt(prompt):
    result_data = []  # Store quant scores for all prompts
    responses = get_openai_responses(prompt)

    prompt_embedding = get_bert_embedding(prompt)  # This returns a list of lists
    response_embeddings = [get_bert_embedding(response) for response in responses]

    cosine_sim_scores = []
    # Create combinations of the prompt embedding and all response embeddings
    embedding_pairs = list(combinations([prompt_embedding] + response_embeddings, 2))
    for pair in embedding_pairs:
        # Since the embeddings are lists, they can be directly used
        score = cosine_similarity([pair[0][0]], [pair[1][0]])[0][0]
        cosine_sim_scores.append(score)

        # Calculate statistical measures
        mean_score = np.mean(cosine_sim_scores)
        median_score = np.median(cosine_sim_scores)
        mode_result = stats.mode(cosine_sim_scores, keepdims=False)


        # Check if the mode result is a scalar or an array and extract the mode value
        if np.isscalar(mode_result.mode):
            mode_score = mode_result.mode
        else:
            mode_score = mode_result.mode[0] if mode_result.mode.size else None
        average_score = np.average(cosine_sim_scores)
        sample_size = len(cosine_sim_scores)

        quant_scores = {
            "mean": mean_score,
            "median": median_score,
            "mode": mode_score,
            "average": average_score,
            "sample_size": sample_size
        }

        result_data.append({"prompt": prompt, "quant_scores": quant_scores})

    return result_data

Now we will test if we can get replies from the three functions in the module

In [ ]:
#Test Cell For get_openai_responses
my_prompt='What is today?'
my_results=get_openai_responses(my_prompt, num_responses=5)
print(my_prompt)
print(my_results)

In [ ]:
#Test for berkembeding function:
my_result2=get_bert_embedding(my_results[0])
print(my_result2)

In [57]:
#Test Cell
my_prompt='What is today?'
my_results3=process_prompt(my_prompt)
print(my_prompt)
print(my_results3)

What is today?
[{'prompt': 'What is today?', 'quant_scores': {'mean': 0.724286940667646, 'median': 0.724286940667646, 'mode': 0.724286940667646, 'average': 0.724286940667646, 'sample_size': 1}}, {'prompt': 'What is today?', 'quant_scores': {'mean': 0.7189585978845091, 'median': 0.7189585978845091, 'mode': 0.7136302551013722, 'average': 0.7189585978845091, 'sample_size': 2}}, {'prompt': 'What is today?', 'quant_scores': {'mean': 0.6847100118650652, 'median': 0.7136302551013722, 'mode': 0.6162128398261774, 'average': 0.6847100118650652, 'sample_size': 3}}, {'prompt': 'What is today?', 'quant_scores': {'mean': 0.6252831393121462, 'median': 0.6649215474637749, 'mode': 0.44700252165338916, 'average': 0.6252831393121462, 'sample_size': 4}}, {'prompt': 'What is today?', 'quant_scores': {'mean': 0.6434774106252421, 'median': 0.7136302551013722, 'mode': 0.44700252165338916, 'average': 0.6434774106252421, 'sample_size': 5}}, {'prompt': 'What is today?', 'quant_scores': {'mean': 0.636637906386038

Testing has been completed and all functions are working as expected. This next cell block will test for version info of our modules.
These shall be added to a requirement txt for a dockerfile.

In [62]:
# Import the required modules
import transformers
import torch
import sklearn
import numpy as np
import scipy
import itertools
import time
import random
import os
from dotenv import load_dotenv

# Print the versions
print(f"transformers version: {transformers.__version__}")
print(f"torch version: {torch.__version__}")
print(f"sklearn version: {sklearn.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"itertools is a built-in module, so it doesn't have a version.")
print(f"time is a built-in module, so it doesn't have a version.")
print(f"random is a built-in module, so it doesn't have a version.")
print(f"os is a built-in module, so it doesn't have a version.")
print(f"dotenv version: {load_dotenv.__module__}")  # For dotenv, we use a different approach

# The OpenAI module version might need a different approach since it's often installed via pip
# and doesn't always have a __version__ attribute. We use pkg_resources to get the version.
try:
    import pkg_resources
    openai_version = pkg_resources.get_distribution("openai").version
    print(f"openai version: {openai_version}")
except Exception as e:
    print(f"Could not determine openai version: {e}")

transformers version: 4.36.2
torch version: 2.1.2+cpu
sklearn version: 1.3.2
numpy version: 1.24.4
scipy version: 1.10.1
itertools is a built-in module, so it doesn't have a version.
time is a built-in module, so it doesn't have a version.
random is a built-in module, so it doesn't have a version.
os is a built-in module, so it doesn't have a version.
dotenv version: dotenv.main
openai version: 1.6.1
